In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *
import sqlite3

Using TensorFlow backend.


In [102]:
conn=sqlite3.connect('harsha.db')
conn.execute('''CREATE TABLE IF NOT EXISTS person (name TEXT(20) NOT NULL);''')
database={}

In [130]:
def loadtodict():
    data=conn.execute('''SELECT name FROM person''')
    data=data.fetchall()
    for row in data:
        print(row[0])
        database[row[0]]=np.load("data/"+row[0]+".npy")

In [121]:
def addperson(name,encoding):
    temp="insert into person(name) values('"+name+"');"
    conn.execute(temp)
    np.save("data\\"+name,encoding)
    print("inserted succesfully")

In [131]:
def delperson(name):
    temp="delete from person where name='"+name+"';"
    conn.execute(temp)
    database.pop(name)
    os.remove("data//"+name+".npy")

In [134]:
addperson("somraj",img_to_encoding("images/mb_2_short.jpg", FRmodel))

inserted succesfully


In [135]:
loadtodict()

harsha
vasanth
mahesh
somraj


In [136]:
database

{'harsha': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
 'mahesh': array([[ 0.1250653 , -0.0556937 , -0.00679408, -0.01854028,  0.11150424,
          0.14437869,  0.0509963 , -0.00331598, -0.07445325, -0.17840335,
         -0.08443446,  0.08988991,  0.09026071, -0.02193167,  0.05617364,
         -0.08980563, -0.00711438, -0.11998504, -0.10730425,  0.23201032,
         -0.0021798 ,  0.03265815,  0.07950632,  0.12795538, -0.07447501,
         -0.09073825, -0.08451234, -0.09504339, -0.10798505,  0.03960141,
         -0.00386546, -0.0137305 , -0.04733571,  0.14134718, -0.01357306,
          0.06020715,  0.05435631,  0.08091162, -0.12519895, -0.04245959,
          0.05866187, -0.01474472,  0.02066107, -0.0542593 , -0.12359606,
          0.02372205,  0.06912614,  0.05706463, -0.01987478,  0.25536612,
         -0.02515062, -0.18247017,  0.04398672,  0.07249464,  0.16291898,
         -0.0576376 , -0.10129909,  0.13295123, -0.03292836, -0.07330219,
         -0.06694847,  0.14348058,  0.05385822, -0.1887

In [137]:
delperson("somraj")

In [138]:
loadtodict()

harsha
vasanth
mahesh


In [139]:
database

{'harsha': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
 'mahesh': array([[ 0.1250653 , -0.0556937 , -0.00679408, -0.01854028,  0.11150424,
          0.14437869,  0.0509963 , -0.00331598, -0.07445325, -0.17840335,
         -0.08443446,  0.08988991,  0.09026071, -0.02193167,  0.05617364,
         -0.08980563, -0.00711438, -0.11998504, -0.10730425,  0.23201032,
         -0.0021798 ,  0.03265815,  0.07950632,  0.12795538, -0.07447501,
         -0.09073825, -0.08451234, -0.09504339, -0.10798505,  0.03960141,
         -0.00386546, -0.0137305 , -0.04733571,  0.14134718, -0.01357306,
          0.06020715,  0.05435631,  0.08091162, -0.12519895, -0.04245959,
          0.05866187, -0.01474472,  0.02066107, -0.0542593 , -0.12359606,
          0.02372205,  0.06912614,  0.05706463, -0.01987478,  0.25536612,
         -0.02515062, -0.18247017,  0.04398672,  0.07249464,  0.16291898,
         -0.0576376 , -0.10129909,  0.13295123, -0.03292836, -0.07330219,
         -0.06694847,  0.14348058,  0.05385822, -0.1887

In [140]:
conn.commit()
conn.close()

In [6]:
np.set_printoptions(threshold=np.nan)

In [7]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

In [8]:
print("Total Params:", FRmodel.count_params())

Total Params: 3743280


In [9]:
def triplet_loss(y_true, y_pred, alpha=0.2):
    
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    

    # Step 1: Compute the (encoding) distance between the anchor and the positive, you will need to sum over axis=-1
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis=-1)
    # Step 2: Compute the (encoding) distance between the anchor and the negative, you will need to sum over axis=-1
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis=-1)
    # Step 3: subtract the two previous distances and add alpha.
    basic_loss = pos_dist - neg_dist + alpha
    # Step 4: Take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0.0))
    
    return loss

In [10]:
FRmodel.compile(optimizer='adam', loss=triplet_loss, metrics=['accuracy'])

In [11]:
load_weights_from_FaceNet(FRmodel)

In [12]:
harshaencod = img_to_encoding("images/vasanth_test1.jpg", FRmodel)
type(harshaencod)

numpy.ndarray

In [ ]:
database = {}
database["danielle"] = img_to_encoding("images/danielle.png", FRmodel)
database["younes"] = img_to_encoding("images/younes.jpg", FRmodel)
database["tian"] = img_to_encoding("images/tian.jpg", FRmodel)
database["andrew"] = img_to_encoding("images/andrew.jpg", FRmodel)
database["kian"] = img_to_encoding("images/kian.jpg", FRmodel)
database["dan"] = img_to_encoding("images/dan.jpg", FRmodel)
database["sebastiano"] = img_to_encoding("images/sebastiano.jpg", FRmodel)
database["bertrand"] = img_to_encoding("images/bertrand.jpg", FRmodel)
database["kevin"] = img_to_encoding("images/kevin.jpg", FRmodel)
database["felix"] = img_to_encoding("images/felix.jpg", FRmodel)
database["benoit"] = img_to_encoding("images/benoit.jpg", FRmodel)
database["arnaud"] = img_to_encoding("images/arnaud.jpg", FRmodel)
database["vasanth"] = img_to_encoding("images/vasanth_test1.jpg", FRmodel)
database["mb"] = img_to_encoding("images/mb_2_short.jpg", FRmodel)

In [ ]:
def who_is_it(image_path, database, model):
    """
    Implements face recognition for the happy house by finding who is the person on the image_path image.
    
    Arguments:
    image_path -- path to an image
    database -- database containing image encodings along with the name of the person on the image
    model -- your Inception model instance in Keras
    
    Returns:
    min_dist -- the minimum distance between image_path encoding and the encodings from the database
    identity -- string, the name prediction for the person on image_path
    """

    # Step 1: Compute the target "encoding" for the image.
    encoding = img_to_encoding(image_path, model)
    
    # Step 2: Find the closest encoding
    
    # Initializing "min_dist" to a large value, say 100
    min_dist = 100
    
    # Loop over the database dictionary's names and encodings.
    for (name, db_enc) in database.items():
        
        # Computing L2(Euclidean) distance between the target "encoding" and the current "emb"
        # from the database.
        dist = np.linalg.norm(encoding - db_enc)

        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name
        if dist < min_dist:
            min_dist = dist
            identity = name
    
    if min_dist > 0.8:
        print("Not in the database,,,,,,UNAUTHORIZED.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))
        
    return min_dist, identity

In [ ]:
who_is_it("images/frame15.jpg", database, FRmodel)